## 転移学習

### 前回の問題点
前回のハリネズミ・ヤマアラシ識別モデルは、上手く転移学習することができず、ハリネズミをヤマアラシとして認識してしまった

その問題点として以下のようなものがあった

1. ハリネズミとヤマアラシの教師データの数に差がありすぎた
    - 以下のように、ヤマアラシの画像はハリネズミの画像の5倍近くあり、学習には不向きだった
        - 訓練用画像数:
            - ハリネズミ:  98枚
            - ヤマアラシ: 487枚
        - 検証用画像数:
            - ハリネズミ: 40枚
            - ヤマアラシ: 40枚
2. 教師データそのものが誤っている可能性があった
    - 人間が手動で分類しており、教師データそのものの妥当性が割と怪しかった
3. 教師データ量が足りていなかった
4. そもそもVGG-16モデル自体古いモデルであり、精度がそれほど高くない

ここでは、データを増やしたり、モデルそのものを変更するという面倒なことはせず、簡単にできそうな 1, 2 の対策を行い、学習精度が向上するか実験してみる

### 教師データの選別
以下の対応を行い、教師データを選別した

1. ハリネズミとヤマアラシの教師データの数が同一になるように一部データを削除
2. ハリネズミなのかヤマアラシなのか怪しい画像は削除

結果、教師データは以下の数となった

- 訓練用画像数
    - ハリネズミ: 90枚
    - ヤマアラシ: 90枚
- 検証用画像数
    - ハリネズミ: 40枚
    - ヤマアラシ: 40枚

このデータセットを使い、もう一度転移学習を行う

In [1]:
# PyCallを使う
using PyCall

# 自作ライブラリ読み込み
include("./lib/Image.jl")
include("./lib/TorchVision.jl")
using .TorchVision # TorchVisionモジュールのexport変数をそのまま使えるようにする

In [2]:
# 画像をVGG-16入力用に変換する関数を生成
## (img::PyObject, phase::Phase) :: Array{Float32,3}
transform_image = make_transformer_for_vgg16_training()

# ハリネズミとヤマアラシの画像へのファイルパスのリスト作成
make_dataset_list(phase::Phase)::Array{String,1} = begin
    phasestr = typestr(phase)
    hedgehogs = map(
        path -> "./dataset/$(phasestr)/hedgehog/$(path)",
        readdir("./dataset/$(phasestr)/hedgehog/")
    )
    porcupines = map(
        path -> "./dataset/$(phasestr)/porcupine/$(path)",
        readdir("./dataset/$(phasestr)/porcupine/")
    )
    vcat(hedgehogs, porcupines)
end

# 画像のラベルをパスから判定する
## ハリネズミ: 0, ヤマアラシ: 1
make_label(img_path::String, phase::Phase)::Int = begin
    label = match(r"/([^/]+)/[^/]+$", img_path).captures[1]
    label = (label == "hedgehog" ? 0 : 1)
end

make_label (generic function with 1 method)

In [3]:
# 乱数初期化
seed_random!(1234)

# ハリネズミとヤマアラシのデータセット作成
@TorchVision.image_dataset Dataset make_dataset_list transform_image make_label

# データローダー作成
dataloader = DataLoader(Dataset; batch_size=32, shuffle=true)

# 学習済みVGG-16モデルをロード
net = models.vgg16(pretrained=true)

# VGG-16の最後の全結合出力層の出力ユニットを2個に付け替える
## 出力は ハリネズミ=0, ヤマアラシ=1 の2種類分類
set!(net.classifier, 6, torch.nn.Linear(in_features=4096, out_features=2))

# 訓練モードに設定
net.train()

PyObject VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17

In [4]:
# 損失関数の定義
criterion = torch.nn.CrossEntropyLoss()

# 学習させるパラメータ名
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

# 学習させるパラメータを設定
params_to_update = set_params_to_update!(net, update_param_names)

# 最適化手法の設定
optimizer = torch.optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

PyObject SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

In [5]:
# モデル訓練
train_model!(net, dataloaders, optimizer, criterion, num_epochs) = begin
    # epoch数分ループ
    for epoch = 1:num_epochs
        @info "Epoch $(epoch)/$(num_epochs)"
        
        # epochごとの学習と検証のループ
        for phase in [Train, Valid]
            # 未学習時の検証性能を確かめるため、最初の訓練は省略
            """
            if epoch == 1 && phase === Train
                continue
            end
            """
            
            if phase === Train
                net.train() # 訓練モードに
                @info "Train mode"
            else
                net.eval() # 検証モードに
                @info "Valid mode"
            end
            
            epoch_loss, epoch_corrects = train!(net, phase, dataloader, optimizer, criterion)
            
            # epochごとの損失と正解率を表示
            epoch_loss = epoch_loss / dataloaders[phase].dataset.__len__()
            epoch_acc = epoch_corrects / dataloaders[phase].dataset.__len__() * 100
            @info "$(phase) Loss: $(epoch_loss), Acc: $(epoch_acc)"
        end
    end
end

# 学習・検証を実行
train_model!(net, dataloader, optimizer, criterion, 2)

┌ Info: Epoch 1/2
└ @ Main In[5]:5
┌ Info: Train mode
└ @ Main In[5]:18
Progress: 100%|█████████████████████████████████████████| Time: 0:01:05
┌ Info: Train Loss: 0.6989214499791463, Acc: 57.77777777777777
└ @ Main In[5]:29
┌ Info: Valid mode
└ @ Main In[5]:21
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
┌ Info: Valid Loss: 0.521346914768219, Acc: 72.5
└ @ Main In[5]:29
┌ Info: Epoch 2/2
└ @ Main In[5]:5
┌ Info: Train mode
└ @ Main In[5]:18
Progress: 100%|█████████████████████████████████████████| Time: 0:01:04
┌ Info: Train Loss: 0.4562105589442783, Acc: 78.88888888888889
└ @ Main In[5]:29
┌ Info: Valid mode
└ @ Main In[5]:21
Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
┌ Info: Valid Loss: 0.4158574640750885, Acc: 78.75
└ @ Main In[5]:29


In [6]:
# 転移学習結果の確認
net.eval() # 推論モードに設定
inputs = [
    transform_image(Image.open("./data/gahag-0059907781-1.jpg"), Valid), # ハリネズミ画像を検証用に変換
    transform_image(Image.open("./data/publicdomainq-0025120muq.jpg"), Valid), # ヤマアラシ画像を検証用に変換
]
pred = net(torch.Tensor(inputs))

PyObject tensor([[ 1.1600, -0.2584],
        [ 0.0044,  0.9110]])

ラベルは `[ハリネズミ, ヤマアラシ]` と定義したため、上記の予測は `1つ目＝ハリネズミ、2つ目＝ヤマアラシ` という結果を表している

したがって、今回の転移学習は成功である

このように、教師データを単純に増やすだけでなく、逆に減らす（良質なデータを選別する）ことでもディープラーニングの精度を向上させることができると言える